In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from quick_pp.objects import Project

# Load well from saved file
project = "BESR"
project_path = rf"data\04_project\{project}.qppp"
project = Project().load(project_path)
project.get_well_names()

# Quick PP Interpretation

In [ ]:
from quick_pp.lithology.sand_silt_clay import SandSiltClay
from quick_pp.porosity import neu_den_xplot_poro, density_porosity, rho_matrix
from quick_pp.qaqc import badhole_flag, mask_outside_threshold, neu_den_xplot_hc_correction
from quick_pp.saturation import *
from quick_pp.plotter import *
from quick_pp.permeability import *
from quick_pp.ressum import *
from quick_pp.rock_type import rock_typing
from quick_pp.plotter import plotly_log
from quick_pp.utils import sand_flagging, min_max_line

In [ ]:
# Load data
well_name = 'BESAR-4'
well = project.get_well(well_name)
well_data = well.data

min_depth = 2390
max_depth = 4255
well_data = well_data[(well_data.DEPTH >= min_depth) & (well_data.DEPTH <= max_depth)].copy()

# Clean up data
well_data = badhole_flag(well_data)
well_data = mask_outside_threshold(well_data, fill=True)

# Initialize lithology model
args = {
    'litho_model': 'ssc',
    # 'dry_clay_point': (.3, 2.7),
    'silt_line_angle': 116,
    # 'wet_clay_point': (0.45, 2.45),
    'sw_water_salinity': 60000,
    'hc_corr_angle': 50,
    'hc_buffer': 0.1,
    'ressum_cutoffs': dict(
        VSHALE=.5,
        PHIT=0,
        SWT=1
    ),
}
ssc_model = SandSiltClay(**args)
vsand, vsilt, vcld, vclb, (nphi_max_line, rhob_max_line) = ssc_model.estimate_lithology(
    well_data['NPHI'], well_data['RHOB'], model='kuttan_modified', normalize= True
)
args.update(ssc_model.__dict__)
well.update_config(args)  # Save lithology model to well

# Implement hydrocarbon correction
nphihc, rhobhc, hc_flag = neu_den_xplot_hc_correction(
    well_data['NPHI'], well_data['RHOB'], gr=well_data['GR'],
    dry_sand_point=args['dry_sand_point'],
    dry_clay_point=args['dry_clay_point'],
    corr_angle=args['hc_corr_angle'], buffer=args['hc_buffer']
)

# Estimate lithology
ssc_model = SandSiltClay(**args)
vsand, vsilt, vcld, vclb, (nphi_max_line, rhob_max_line) = ssc_model.estimate_lithology(
    nphihc, rhobhc, model='kuttan_modified', normalize= True
)
vclw = vcld + vclb

# Estimate porosity
phit = neu_den_xplot_poro(
    nphihc, rhobhc, model='ssc', reservoir=True,
    dry_sand_point=args['dry_sand_point'],
    dry_silt_point=args['dry_silt_point'],
    dry_clay_point=args['dry_clay_point'],
)

# PHID needs unnormalized lithology
vsand_un, vsilt_un, vcld_un, vclb_un, (nphi_max_line, rhob_max_line) = SandSiltClay(**args).estimate_lithology(
    nphihc, rhobhc, model='kuttan_modified', normalize= False
)
rho_ma = rho_matrix(vsand_un, vsilt_un, vcld_un)
phid = density_porosity(rhobhc, rho_ma)
phie = phit - vclb

# Estimate water saturation
temp_grad = estimate_temperature_gradient(well_data['DEPTH'], 'metric')  # Arbitrarily use MD instead of TVD
water_salinity = args['sw_water_salinity']
rw_salinity = estimate_rw_temperature_salinity(temp_grad, water_salinity)
b = estimate_b_waxman_smits(temp_grad, rw_salinity)
qv = estimate_qv(vcld, phit, cec_clay=.1)

rt_water = estimate_rt_water_trend(well_data['RT'])
# rw = estimate_rw_archie(phit, rt_water)
rw = estimate_rw_waxman_smits(phit, rt_water, B=b, Qv=qv)

# TODO: Implement m estimation/ based on zonation
m = 1.85

swt = waxman_smits_saturation(well_data['RT'], rw, phit, qv)
swt_a = archie_saturation(well_data['RT'], rw, phit, 1, 2, 2)

# Estimate permeability
constant = vclb**1.75
Swirr = constant / phit
perm = choo_permeability(vclw, vsilt, phit)
# perm_kc = kozeny_carman_permeability(phit, Swirr)
# perm_t = timur_permeability(phit, Swirr)
# perm_c = coates_permeability(phit, Swirr)
# perm_tx = tixier_permeability(phit, Swirr)

# Estimate rock types
rock_flag = rock_typing(vclw)

# Update ZONES
well_data = sand_flagging(well_data)
zones = well_data.ZONES

# Estimate reservoir summary
ressum_df = calc_reservoir_summary(well_data.DEPTH, vclw, phit, swt, perm, zones, cutoffs=args['ressum_cutoffs'])


# Update data in the project
well_data['NPHI_HC'] = nphihc
well_data['RHOB_HC'] = rhobhc
well_data['HC_FLAG'] = hc_flag
well_data['VSAND'] = vsand
well_data['VSILT'] = vsilt
well_data['VCLB'] = vclb
well_data['VCLD'] = vcld
well_data['VCLW'] = vclw
well_data['PHIT'] = phit
well_data['PHIE'] = phie  # Temporary
well_data['PHID'] = phid
well_data['RW'] = rw
well_data['B'] = b
well_data['Qv'] = qv
well_data['M'] = m
well_data['SWT'] = swt
# well_data['SWE'] = swe
well_data['PERM'] = perm
well_data['ROCK_FLAG'] = rock_flag


In [ ]:
well_data['NPHI_line'] = nphi_max_line
well_data[['NPHI', 'NPHI_line']].plot(figsize=(15, 3))

In [ ]:
well_data['RHOB_line'] = rhob_max_line
well_data[['RHOB', 'RHOB_line']].plot(figsize=(15, 3))

In [ ]:
copy_args = args.copy()
copy_args.pop('silt_line_angle')
print(copy_args)
neutron_density_xplot(well_data['NPHI'], well_data['RHOB'], **copy_args)

In [ ]:
copy_args = args.copy()
copy_args.pop('silt_line_angle')
print(copy_args)
neutron_density_xplot(nphihc, rhobhc, **copy_args)

In [ ]:
test = well_data.copy()

test['RT_trend'] = estimate_rt_water_trend(test['RT'], alpha=0.05)
test['RW_archie'] = estimate_rw_archie(test['PHIT'], test['RT_trend'], m=2)
test['RW_salinity'] = estimate_rw_temperature_salinity(temp_grad, water_salinity)
test['RW_WS'] = estimate_rw_waxman_smits(test['PHIT'], test['RT_trend'], B=test['B'], Qv=test['Qv'])

test[['RT', 'RT_trend', 'RW_salinity', 'RW_archie', 'RW_WS']].plot(figsize=(20, 3), logy=True)

In [ ]:
# Plot the results
fig = plotly_log(well_data, depth_uom=well.depth_uom)
fig.show()

In [ ]:
from quick_pp.qaqc import quick_qc

test = quick_qc(well_data)

plt.figure(figsize=(15,2))
plt.plot(test['DEPTH'], test['QC_FLAG'])
plt.show()

plt.figure(figsize=(15,3))
plt.plot(test['DEPTH'], test.SWT, label='SWT')
plt.plot(test['DEPTH'], test.SAND_FLAG, label='SAND_FLAG')
plt.legend()
plt.show()

plt.figure(figsize=(15,3))
plt.plot(test['DEPTH'], test.PHIT, label='PHIT')
# plt.plot(test['DEPTH'], test.PHID, label='PHID')
plt.legend()
plt.show()

plt.figure(figsize=(15,3))
plt.plot(test['DEPTH'], test.VCLW, label='VSHALE')
plt.plot(test['DEPTH'], test.VSH_GR, label='VSH_GR')
plt.legend()
plt.show()

In [ ]:
well = project.get_well(well_name)
well_data = well.data

min_depth = 2390
max_depth = 4255
well_data = well_data[(well_data.DEPTH >= min_depth) & (well_data.DEPTH <= max_depth)].copy()

# Update ZONES
test = sand_flagging(well_data)

min = 2800
max = 3150
flag_df = test[(test.DEPTH > min) & (test.DEPTH < max)].copy()
plt.figure(figsize=(15,2))
plt.plot(flag_df['DEPTH'], flag_df['ZONES'])
plt.plot(flag_df['DEPTH'], flag_df['SAND_FLAG'], label='final')
plt.plot(flag_df['DEPTH'], flag_df['VCLW'])
plt.legend()

In [ ]:
# # Save the results to the well
# well.update_data(well_data)
# well.update_ressum(ressum_df)
# project.save_well(well)

In [ ]:
# project.save()